# Rand Pruned Monte-Carlo

In [2]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
import random
import time
from collections import deque
from InfMaxProblem import Monte_Carlo as mc

%matplotlib inline

In [3]:
# 入力は隣接リスト(numpy)[[from_node, to_node, edge_prob],...]
def live_edge_graph_edges(p, p_len):
    rand = np.random.uniform(0, 1, p_len)
    prob = p.T[2]
    l = np.where(rand < prob)[0]
    return np.array([[p[i][0], p[i][1]] for i in l])

# %time live_edge_graph_edges(network_np, len(network_np))
# Wall time: 86.1 ms

In [4]:
# 幅優先探索によって、Sから到達可能なノードを返す
def bfs(G, S):
    visited = {s:None for s in S}
    queue = deque(S)
    while queue:
        v = queue.popleft()
        out_node = G.successors(v)
        for u in out_node:
            if not (u in visited):
                queue.append(u)
                visited[u] = v
    return visited

In [5]:
# 幅優先探索によって、Sから到達可能なノードを返す
def bfs_reverse(G, S):
    visited = {s:None for s in S}
    queue = deque(S)
    while queue:
        v = queue.popleft()
        in_node = G.predecessors(v)
        for u in in_node:
            if not (u in visited):
                queue.append(u)
                visited[u] = v
    return visited

In [6]:
# 強連結成分分解に使うための関数
def dfs_go(G):
    visited = dict([])
    # vs:帰りがけ順
    vs = deque([])
    for s in G.nodes():
        if s not in visited:
            visited[s] = None
            Stack = deque([s])
            vs_tmp = deque([s])
            while Stack:
                v = Stack.pop()
                #if v not in vs_tmp:
                vs_tmp.appendleft(v)
                out_node = G[v]
                for u in out_node:
                    if u not in visited:
                        visited[u] = s
                        Stack.append(u)
            #vs = vs_tmp + vs
            vs.extendleft(vs_tmp)
    return visited, vs

# 強連結成分分解に使うための関数
def dfs_back(G, vs):
    group = dict([])
    group_num = 0
    DAG = nx.DiGraph()
    for s in vs:
        if s not in group:
            w = 1
            Stack = deque([s])
            members = []
            while Stack:
                v = Stack.pop()
                members.append(v)
                group[v] = group_num
                in_node = G.predecessors(v)
                for u in in_node:
                    if u not in group:
                        group[u] = group_num
                        w += 1
                        Stack.append(u)
                    else:
                        if group_num !=  group[u]:
                            DAG.add_edge(group[u], group_num)
            DAG.add_node(group_num, weight=w, members=members)
            group_num += 1
    return group, DAG

def scc_DAG(G):
    visited ,vs = dfs_go(G)
    group, DAG = dfs_back(G, vs)
    return group, DAG

In [7]:
# i回目のシュミレーションでv_Vが到達できる頂点数
def GAIN(i, v_V, comp, G, A, S, h, D, V):
    if v_V not in comp[i]:
        return 0        
        
    # v:i回目のシュミレーションで作成されたグラフのv_Vを含む強連結成分
    v = comp[i][v_V]
    
    # V[i]にvがない場合0(後に消されていくため)
    if v not in V:
        delta[i][v] = 0
        return 0
    
    # delta
    if latest[i][v]:
        return delta[i][v]
        
    latest[i][v] = True
    
    # len(S)==0の理由は初回のみ行えば良いため(h(ハブ)以降の到達頂点数は一回行えば十分であるため)
    # vがhのacestorだった場合、hの到達頂点数を計算して、他のacestorの時にも使い回す
    if (v in A[i]) and (len(S) == 0):
        # GAINの引数は後で変える
        # hのGAINをはじめから足しておく
        h_Vs = G[i].nodes[h[i]]["members"]
        delta[i][v] = GAIN(i, h_Vs[0], comp, G, A, S, h, D, V)
    else:
        delta[i][v] = 0
    
    # bfs
    Q = deque([v])
    # Xは探索済みの強連結成分
    X = set([v])
    while Q:
        u = Q.popleft()
        
        if (v in A[i]) and (u in D[i]) and (len(S) == 0):
            continue
            
        delta[i][v] += G[i].nodes[u]["weight"]
        
        
        Edges = G[i].out_edges(u)
        for u_, w in Edges:
            # 探索済みの強連結成分は探索しなくていいので、w not in X
            # w in V[i]はのちのupdateでV[i]が変化するため
            if (w not in X) and (w in V):
                Q.append(w)
                X.add(w)
        
    return delta[i][v]

In [8]:
def UPDATEDAG(i, t_V, comp, G):
    if t_V not in comp[i]:
        return G[i]
    
    # t:DAG上でのノードid
    t = comp[i][t_V]
    
    if t in G[i]:
        # t -> u
        u = list(bfs(G[i], [t]))
        # u = list(dict(nx.bfs_edges(G[i], t)))
        # v -> u:上で求めたuにだどりつくvを求める
        v = set(bfs_reverse(G[i], u))

        # v かつ Viに存在する頂点
        v_ = list(v & set(G[i].nodes()))
        latest[i].update(zip(v_, [False]*len(v_)))

        G[i].remove_nodes_from(u)
        return G[i]
    else:
        return G[i]

In [53]:
def RPMC_greedy(network_np, k, R, eps):
    time_list = []
    # E_ = dict([])
    # G_ = dict([])
    G = dict([])
    comp = dict([])
    h = dict([])
    D = dict([])
    A = dict([])
    V = dict([])
    global latest
    latest = dict([])
    global delta
    delta = {i:dict([]) for i in range(R)}
    
    # 全体のグラフ
    Node = np.unique(network_np.T[[0,1]])
    
    # 元グラフの頂点集合
    G_V = [int(node) for node in Node]
    
    # copy用に作っておく
    G_copy = nx.DiGraph()
    G_copy.add_nodes_from(G_V)
    
    for i in tqdm(range(R)):
        
        E_ = live_edge_graph_edges(network_np, len(network_np))
        
        G_ = G_copy.copy()
        G_.add_edges_from(E_)
        
        comp[i], G[i] = scc_DAG(G_)
        
        G_i_deg = dict(G[i].degree())
        h[i] = max(G_i_deg, key=G_i_deg.get)
        
        D[i] = set(bfs(G[i], [h[i]]))
        
        A[i] = set(bfs_reverse(G[i], [h[i]])) - set([h[i]])
        
        V[i] = G[i].nodes()
        latest[i] = {v:False for v in V[i]}
    print("comp init")
    
    S = []
    # サンプリング回数
    s = int((len(G_V) / k) * np.log(1/eps))
    for j in range(k):
        start = time.time()
        RS = np.random.choice(list(set(G_V) - set(S)), s)
        
        v_gain = {v:sum([GAIN(i, v, comp, G, A, S, h, D, G[i].nodes()) for i in range(R)])/R for v in tqdm(RS)}

        t = max(v_gain, key=v_gain.get)
        
        S.append(t)
        
        for i in range(R):
            G[i] = UPDATEDAG(i, t, comp, G)
        time_list.append(time.time())
        
    return S

In [54]:
# 枝確率を計算済みのネットワークを読み込む
network = pd.read_csv("data.csv")
network.head()

,# FromNodeId,ToNodeId,WC
0,0,4,0.015625
1,0,5,0.011236
2,0,7,0.100000
3,0,8,0.031250
4,0,9,0.111111


In [55]:
# numpy型に変換
network_np = network.values

In [56]:
k = 50
R = 50
eps = 0.1
%timeit RPMC_greedy(network_np, k, R, eps)

/home/fueki/anaconda3/envs/all/lib/python3.7/site-packages/ipykernel_launcher.py:26: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



comp init


/home/fueki/anaconda3/envs/all/lib/python3.7/site-packages/ipykernel_launcher.py:53: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



comp init



comp init


...


comp init



comp init



comp init



comp init



2.36 s ± 65.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
# G = nx.DiGraph()
# G.add_weighted_edges_from(network_np)

In [12]:
# %time mc.approx_inf_size_IC_T(G, [0, 1, 29, 27, 26], 10000)

In [13]:
# # 枝確率を計算済みのネットワークを読み込む
# network = pd.read_csv("data/Epinions/WC_small.csv")
# network.head()

# # numpy型に変換
# network_np = network.values

# 計算実験

In [79]:
def RPMC_greedy_time(network_np, k, R, eps):
    time_list = []
    # E_ = dict([])
    # G_ = dict([])
    G = dict([])
    comp = dict([])
    h = dict([])
    D = dict([])
    A = dict([])
    V = dict([])
    global latest
    latest = dict([])
    global delta
    delta = {i:dict([]) for i in range(R)}
    G_V = set([])
    
    for i in tqdm(range(R)):
        
        E_ = live_edge_graph_edges(network_np, len(network_np))
        
        # 枝が伸びている頂点集合
        G_V |= set(np.unique(E_.T[[0,1]]))
        
        G_ = nx.DiGraph()
        G_.add_edges_from(E_)
        
        comp[i], G[i] = scc_DAG(G_)
        
        G_i_deg = dict(G[i].degree())
        h[i] = max(G_i_deg, key=G_i_deg.get)
        
        D[i] = set(bfs(G[i], [h[i]]))
        
        A[i] = set(bfs_reverse(G[i], [h[i]])) - set([h[i]])
        
        V[i] = G[i].nodes()
        latest[i] = {v:False for v in V[i]}
    print("comp init")
    
    S = []
    # サンプリング回数
    s = int((len(G_V) / k) * np.log(1/eps))
    for j in tqdm(range(k)):
        start = time.time()
        RS = np.random.choice(list(set(G_V) - set(S)), s)
        v_gain = {v:sum([GAIN(i, v, comp, G, A, S, h, D, G[i].nodes()) for i in range(R)])/R for v in RS}
             
        t = max(v_gain, key=v_gain.get)
        
        S.append(t)
        
        for i in range(R):
            G[i] = UPDATEDAG(i, t, comp, G)
        time_list.append(time.time()-start)
        
    return S, time_list

## Epinions

### WC

In [80]:
# 枝確率を計算済みのネットワークを読み込む
network = pd.read_csv("data/Epinions/WC.csv")
network.head()

,# FromNodeId,ToNodeId,WC
0,0,4,0.008000
1,0,5,0.005682
2,0,7,0.033333
3,0,8,0.009615
4,0,9,0.066667


In [81]:
# numpy型に変換
network_np = network.values

In [82]:
run_time = dict([])
S = dict([])
run_time_k = dict([])

EPS = [0.5, 0.1, 0.01]
R = 200
k = 50
# 実験回数
EXP = 10
for eps in EPS:
    for i in range(EXP):
        start = time.time()
        S[str(eps) + "_" + str(i)], run_time_k[str(eps) + "_" + str(i)] = RPMC_greedy_time(network_np, k, R, eps)
        run_time[str(eps) + "_" + str(i)] = time.time() - start

/home/fueki/anaconda3/envs/all/lib/python3.7/site-packages/ipykernel_launcher.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



comp init


/home/fueki/anaconda3/envs/all/lib/python3.7/site-packages/ipykernel_launcher.py:46: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



comp init



comp init


...


comp init



comp init



comp init



comp init



comp init


In [87]:
pd.DataFrame(S).to_csv("./result/Epinions/RPMC/seed.csv", index=False)

In [89]:
pd.DataFrame(run_time_k).to_csv("./result/Epinions/RPMC/seed_time.csv", index=False)

In [92]:
pd.DataFrame([run_time]).to_csv("./result/Epinions/RPMC/run_time.csv", index=False)